<a href="https://colab.research.google.com/github/RinadAkel/Sentiment-Analysis/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

© 2021 Zaka AI, Inc. All Rights Reserved

# Text classification

Text classification is one of the important tasks of text mining

![alt text](http://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1535125878/NLTK3_zwbdgg.png)

In this notebook, we will perform Sentiment Analysis on IMDB movies reviews. Sentiment Analysis is the art of extracting people's opinion from digital text. We will use a regression model from Scikit-Learn able to predict the sentiment given a movie review. 

We will use [the IMDB movie review dataset](http://ai.stanford.edu/~amaas/data/sentiment/), which consists of 50,000 movies review (50% are positive, 50% are negative).

* [Numpy](http://www.numpy.org/) — a package for scientific computing.
* [Pandas](https://pandas.pydata.org/) — a library providing high-performance, easy-to-use data structures and data analysis tools for the Python
* [Matplotlib](https://matplotlib.org/) — a package for plotting & visualizations.
* [scikit-learn](http://scikit-learn.org/stable/index.html) — a tool for data mining and data analysis.
* [NLTK](http://www.nltk.org/) — a platform to work with natural language.

##Loading the data

### Importing the libraries and necessary dictionaries

In [ ]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from tensorflow import keras

# download Punkt Sentence Tokenizer
nltk.download('punkt')
# download stopwords
nltk.download('stopwords')

### Loading the dataset in our directory

In [ ]:
# download IMDB dataset
!wget "https://raw.githubusercontent.com/javaidnabi31/Word-Embeddding-Sentiment-Classification/master/movie_data.csv" -O "movie_data.csv"  

# list files in current directory
!ls -lah  

###Reading the dataset file and getting info on it
Using pandas to read the csv file and displaying the first 10 rows

In [ ]:
# the path to the IMDB dataset
dataset_path = 'movie_data.csv'  

# read file (dataset) into our program using pandas
data = pd.read_csv(dataset_path) 

# display first 10 rows
data.head(10)  

Getting info on our dataset

In [ ]:
data.info()

A balanced dataset in sentiment analysis is a dataset which holds an equal amount of positive sentiment data and negative sentiment data, meaning 50% of the data is positive and 50% is negative

In [ ]:
# check if dataset is balanced (number of positive sentiment = number of negative sentiment) 
# by plotting the different classes
data.sentiment.value_counts().plot(kind='bar')

## Text cleaning

In [ ]:
print(data.review[10])

Let's define a function that would clean each movie review (sentence)

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

english_stopwords = stopwords.words('english')
stemmer = PorterStemmer()

def clean_review(text):
  # convert to lower case
  text = text.lower()

  # remove none alphabetic characters
  text = re.sub(r'[^a-z]', ' ', text)

  # stem words 
  # split into words
  tokens = word_tokenize(text)

  # stemming of words
  stemmed = [stemmer.stem(word) for word in tokens]

  text = ' '.join(stemmed)

  # remove stopwords
  text = ' '.join([word for word in text.split() if word not in english_stopwords])

  return text

And try it out on an instance of the dataset

In [ ]:
print(data['review'][1])
print(clean_review(data['review'][1]))

And now clean the entire dataset reviews

In [ ]:
# apply to all dataset
data['clean_review'] = data['review'].apply(clean_review)
data.head() 

## Split dataset for training and testing
We will split our data into two subsets: a 50% subset will be used for training the model for prediction and the remaining 50% will be used for evaluating or testing its performance. The random state ensures reproducibility of the results.

In [ ]:
from sklearn.model_selection import train_test_split
 
X = data['clean_review'].values
y = data['sentiment'].values

# Split data into 50% training & 50% test
# let's all use a random state of 42 for example to ensure having the same split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

## Feature extraction with Bag of Words
In this section, let's apply the Bag of Words method to learn the vocabulary of our text and with it transform our training input data.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# define a CountVectorizer (with binary=True and max_features=10000)
vectorizer = CountVectorizer(binary=True, max_features=10000)  

# learn the vocabulary of all tokens in our training dataset
vectorizer.fit(x_train)  

# transform x_train to bag of words
x_train_bow = vectorizer.transform(x_train)  
x_test_bow = vectorizer.transform(x_test)   

print(x_train_bow.shape, y_train.shape)
print(x_test_bow.shape, y_test.shape)

## Classification
Our data is ready for classification. Let's use [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [ ]:
from sklearn.linear_model import LogisticRegression

# define the LogisticRegression classifier
model = LogisticRegression()

# train the classifier on the training data
model.fit(x_train_bow, y_train)

# get the mean accuracy on the training data
acc_train = model.score(x_train_bow, y_train)  

print('Training Accuracy:', acc_train)

Evaluating the performance of our model through its accuracy score

In [ ]:
# Evaluate model with test data

acc_test = model.score(x_test_bow, y_test)  
print('Test Accuracy:', acc_test)

## Let's use the model to predict!
To do so, let's create a predict function which takes as argument our model and the bag of words vectorizer together with a review on which it would predict the sentiment. This review should be cleaned with the `clean_review` function we built, transformed by bag of words and then used for prediction with `model.predict()`.

In [ ]:
# define predict function
def predict(model, vectorizer, review):
    review = clean_review(review)
    review_bow = vectorizer.transform([review])
    return model.predict(review_bow)[0]

And let's try it out on an example

In [ ]:
review = 'The movie was great!!!'
predict(model, vectorizer, review)

In [ ]:
review2 = 'the film was not Bad'
predict(model, vectorizer, review2)